<a href="https://colab.research.google.com/github/renaldyh27/Analysis-on-Trips-and-Drivers/blob/main/wip_fpc_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import scipy.stats as stats

# Importing and Processing Data

In [14]:
#Import data and select the columns
data = pd.read_csv('WIP_data.csv')
data = data[['MTC_ISS_TRKG_ID', 'MITG_CRT_LTS', 'MinOfMITG_AUDT_LTS', 'MITG_CL_LTS', 'MITG_ISS_TYP_TXT', 'MITG_PRI_CD']]

#Filter to only FPC data
data = data.loc[data['MITG_ISS_TYP_TXT'] == 'FPC']

In [15]:
df = data.rename(columns={'MTC_ISS_TRKG_ID': 'ID','MITG_CRT_LTS' : 'creation_time', 'MITG_CL_LTS': 'closing_time', 'MinOfMITG_AUDT_LTS' : 'audit_time', 'MITG_ISS_TYP_TXT' : 'iss_type', 'MITG_PRI_CD' : 'req_type'})
df.dropna(subset=['closing_time'], inplace=True)
#Adding column for time it takes for issues to be closed (completion_time)
df['closing_time'] = pd.to_datetime(df['closing_time'])
df['creation_time'] = pd.to_datetime(df['creation_time'])
df['audit_time'] = pd.to_datetime(df['audit_time'])
df['completion_time'] = df['closing_time'] - df['creation_time']
df['cmp_time_hr'] = df['completion_time'].apply(lambda x : x.total_seconds())/60/60
df['crt_mth_yr'] = [d.strftime('%b %Y') for d in df['creation_time']]
df = df.loc[df['crt_mth_yr'].str.contains('2019') == False] #Removing 2019 since not enough data

#Removing the nulls
df.dropna()


df

,ID,creation_time,audit_time,closing_time,iss_type,req_type,completion_time,cmp_time_hr,crt_mth_yr
288,1654845,2020-01-01 00:29:00,NaT,2020-01-02 11:57:00,FPC,Misc,1 days 11:28:00,35.466667,Jan 2020
291,1654874,2020-01-01 02:27:00,NaT,2020-01-07 12:44:00,FPC,Expedite,6 days 10:17:00,154.283333,Jan 2020
292,1654883,2020-01-01 03:07:00,NaT,2020-01-09 06:16:00,FPC,Misc,8 days 03:09:00,195.150000,Jan 2020
294,1654958,2020-01-01 03:56:00,NaT,2020-01-15 07:36:00,FPC,Misc,14 days 03:40:00,339.666667,Jan 2020
296,1655002,2020-01-01 05:01:00,NaT,2020-01-07 12:52:00,FPC,Misc,6 days 07:51:00,151.850000,Jan 2020
...,...,...,...,...,...,...,...,...,...
57965,1912837,2021-07-16 07:47:00,NaT,2021-07-16 10:19:00,FPC,Misc,0 days 02:32:00,2.533333,Jul 2021
57975,1912897,2021-07-16 10:01:00,NaT,2021-07-16 16:34:00,FPC,Misc,0 days 06:33:00,6.550000,Jul 2021
57979,1912902,2021-07-16 10:11:00,NaT,2021-07-16 16:37:00,FPC,Misc,0 days 06:26:00,6.433333,Jul 2021
57980,1912913,2021-07-16 10:26:00,NaT,2021-07-16 16:40:00,FPC,Misc,0 days 06:14:00,6.233333,Jul 2021


## Remove Outliers

In [19]:
# If completion time is 2.5 standard deviations away, we'll remove them
df = df[(np.abs(stats.zscore(df['cmp_time_hr']))<2.5)] 
df

,ID,creation_time,audit_time,closing_time,iss_type,req_type,completion_time,cmp_time_hr,crt_mth_yr
288,1654845,2020-01-01 00:29:00,NaT,2020-01-02 11:57:00,FPC,Misc,1 days 11:28:00,35.466667,Jan 2020
291,1654874,2020-01-01 02:27:00,NaT,2020-01-07 12:44:00,FPC,Expedite,6 days 10:17:00,154.283333,Jan 2020
292,1654883,2020-01-01 03:07:00,NaT,2020-01-09 06:16:00,FPC,Misc,8 days 03:09:00,195.150000,Jan 2020
294,1654958,2020-01-01 03:56:00,NaT,2020-01-15 07:36:00,FPC,Misc,14 days 03:40:00,339.666667,Jan 2020
296,1655002,2020-01-01 05:01:00,NaT,2020-01-07 12:52:00,FPC,Misc,6 days 07:51:00,151.850000,Jan 2020
...,...,...,...,...,...,...,...,...,...
57965,1912837,2021-07-16 07:47:00,NaT,2021-07-16 10:19:00,FPC,Misc,0 days 02:32:00,2.533333,Jul 2021
57975,1912897,2021-07-16 10:01:00,NaT,2021-07-16 16:34:00,FPC,Misc,0 days 06:33:00,6.550000,Jul 2021
57979,1912902,2021-07-16 10:11:00,NaT,2021-07-16 16:37:00,FPC,Misc,0 days 06:26:00,6.433333,Jul 2021
57980,1912913,2021-07-16 10:26:00,NaT,2021-07-16 16:40:00,FPC,Misc,0 days 06:14:00,6.233333,Jul 2021


In [20]:
fig = px.scatter(df, x="creation_time", y="cmp_time_hr", 
                color="req_type",
                title='Issue Completion Times Grouped by Request Type'
                 )
fig.show()

In [21]:
# Get statistics for each month of 2020 and 2021
monthly_mean = df.groupby('crt_mth_yr')['cmp_time_hr'].mean().reset_index().rename(columns={'cmp_time_hr': 'mean_cmp_time_month'})
monthly_median = df.groupby('crt_mth_yr')['cmp_time_hr'].median().reset_index().rename(columns={'cmp_time_hr': 'median_cmp_time_month'})

df = pd.merge(df, monthly_mean, on='crt_mth_yr')
df = pd.merge(df, monthly_median, on='crt_mth_yr')


In [22]:
monthly_std = df.groupby('crt_mth_yr')['cmp_time_hr'].std().reset_index().rename(columns={'cmp_time_hr': 'std_cmp_time_month'})
df = pd.merge(df, monthly_std, on='crt_mth_yr')

In [25]:
df

,ID,creation_time,audit_time,closing_time,iss_type,req_type,completion_time,cmp_time_hr,crt_mth_yr,mean_cmp_time_month,median_cmp_time_month,std_cmp_time_month
0,1654845,2020-01-01 00:29:00,NaT,2020-01-02 11:57:00,FPC,Misc,1 days 11:28:00,35.466667,Jan 2020,50.877536,34.025,59.783346
1,1654874,2020-01-01 02:27:00,NaT,2020-01-07 12:44:00,FPC,Expedite,6 days 10:17:00,154.283333,Jan 2020,50.877536,34.025,59.783346
2,1654883,2020-01-01 03:07:00,NaT,2020-01-09 06:16:00,FPC,Misc,8 days 03:09:00,195.150000,Jan 2020,50.877536,34.025,59.783346
3,1654958,2020-01-01 03:56:00,NaT,2020-01-15 07:36:00,FPC,Misc,14 days 03:40:00,339.666667,Jan 2020,50.877536,34.025,59.783346
4,1655002,2020-01-01 05:01:00,NaT,2020-01-07 12:52:00,FPC,Misc,6 days 07:51:00,151.850000,Jan 2020,50.877536,34.025,59.783346
...,...,...,...,...,...,...,...,...,...,...,...,...
8502,1912837,2021-07-16 07:47:00,NaT,2021-07-16 10:19:00,FPC,Misc,0 days 02:32:00,2.533333,Jul 2021,29.954027,14.150,42.511255
8503,1912897,2021-07-16 10:01:00,NaT,2021-07-16 16:34:00,FPC,Misc,0 days 06:33:00,6.550000,Jul 2021,29.954027,14.150,42.511255
8504,1912902,2021-07-16 10:11:00,NaT,2021-07-16 16:37:00,FPC,Misc,0 days 06:26:00,6.433333,Jul 2021,29.954027,14.150,42.511255
8505,1912913,2021-07-16 10:26:00,NaT,2021-07-16 16:40:00,FPC,Misc,0 days 06:14:00,6.233333,Jul 2021,29.954027,14.150,42.511255


In [26]:
df[['crt_mth_yr', 'mean_cmp_time_month', 'median_cmp_time_month']]

,crt_mth_yr,mean_cmp_time_month,median_cmp_time_month
0,Jan 2020,50.877536,34.025
1,Jan 2020,50.877536,34.025
2,Jan 2020,50.877536,34.025
3,Jan 2020,50.877536,34.025
4,Jan 2020,50.877536,34.025
...,...,...,...
8502,Jul 2021,29.954027,14.150
8503,Jul 2021,29.954027,14.150
8504,Jul 2021,29.954027,14.150
8505,Jul 2021,29.954027,14.150


In [29]:
fig = px.bar(df[['crt_mth_yr', 'mean_cmp_time_month', 'median_cmp_time_month']].drop_duplicates(), 
                x='crt_mth_yr', 
                y='mean_cmp_time_month',
                title='Mean Completion Time Over the Months 2020-2021',
                barmode='group')
fig.show()

Discuss what our goals are, by how much do we want to improve completion time perhaps?

## Analyze by Req Type Group

In [ ]:
# Separate statistics by grouping based on req type
df.groupby('req_type')['cmp_time_hr'].describe()

,count,mean,std,min,25%,50%,75%,max
req_type,,,,,,,,
AOG,26.0,134.214103,161.172521,0.500000,14.791667,123.175000,173.658333,649.616667
Expedite,204.0,136.297059,137.899402,1.566667,56.829167,98.783333,172.583333,1088.900000
Misc,8058.0,103.482229,157.015597,0.016667,21.404167,68.191667,152.833333,9265.716667
PROJECT,295.0,259.341017,487.138875,12.500000,84.125000,119.000000,230.850000,4926.633333


In [33]:
# Get statistics for each month of 2020 and 2021
monthly_mean_rtype = df.groupby(['crt_mth_yr', 'req_type'])['cmp_time_hr'].mean().reset_index().rename(columns={'cmp_time_hr': 'mean_cmp_time_month_rtype'})
monthly_median_rtype = df.groupby(['crt_mth_yr', 'req_type'])['cmp_time_hr'].median().reset_index().rename(columns={'cmp_time_hr': 'median_cmp_time_month_rtype'})

df_rtype = pd.merge(df, monthly_mean_rtype, on=['crt_mth_yr', 'req_type'])
df_rtype = pd.merge(df_rtype, monthly_median_rtype, on=['crt_mth_yr', 'req_type'])


fig = px.bar(df_rtype[['crt_mth_yr', 'req_type', 'mean_cmp_time_month_rtype']].drop_duplicates(), 
                x='crt_mth_yr', 
                y='mean_cmp_time_month_rtype',
                title='Mean Completion Time Over the Months 2020-2021 Grouped by Req Type',
                barmode='group',
                color='req_type')
fig.show()




```
# This is formatted as code
```

# Weekly Metrics

# Import FPC Historic Data

In [ ]:
#Our dataset contains data starting from Jan 2019 until Jul 2021
data = pd.read_csv("IMT_history_July2021.csv")
data.drop(data.columns[0], axis=1, inplace=True) # Drop extra column
fpc_df = data.loc[data['MITG_ISS_TYP_TXT'] == 'FPC                 ']
fpc_df['date'] = fpc_df['MITG_CRT_LTS'].str.slice(stop = 10)
fpc_df

c:\Users\Renaldy\Documents\Delta\imtenv\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning:

Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.

c:\Users\Renaldy\Documents\Delta\imtenv\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,MTC_ISS_TRKG_ID,APCN_CATG_CDE,MITG_ISS_TYP_TXT,MITG_CRT_LTS,AC_NB,date
3,1400382,CSR,FPC,2019-01-01 00:31:29.229199,3844.0,2019-01-01
4,1400383,CSR,FPC,2019-01-01 00:32:25.086757,3260.0,2019-01-01
5,1400384,CSR,FPC,2019-01-01 00:33:29.215995,3318.0,2019-01-01
35,1400414,CSR,FPC,2019-01-01 01:53:29.088738,6716.0,2019-01-01
36,1400415,CSR,FPC,2019-01-01 01:54:45.109816,189.0,2019-01-01
...,...,...,...,...,...,...
522246,1922695,CSR,FPC,2021-08-02 11:37:47.532743,06702,2021-08-02
522248,1922718,CSR,FPC,2021-08-02 12:01:55.885598,06705,2021-08-02
522278,1922734,CSR,FPC,2021-08-02 12:39:56.438301,03021,2021-08-02
522336,1922787,CSR,FPC,2021-08-02 13:25:15.962335,09524,2021-08-02


In [ ]:
# Groupby count of overall IMT per day and label it as 'y' since it's the required format to be fed to the Prophet model
countdf = fpc_df.groupby(['date'], as_index=False).MTC_ISS_TRKG_ID.count().rename(columns = {'MTC_ISS_TRKG_ID':'y'})
fpc_df = pd.merge(fpc_df, countdf, on='date')

#Convert to datetime and get the hour, time period, day, day of the week, month, and year
fpc_df['date'] = pd.to_datetime(fpc_df.date)

fpc_df['month'] = fpc_df['date'].dt.month
fpc_df['year'] = fpc_df['date'].dt.year
fpc_df['day'] = fpc_df['date'].dt.day
fpc_df['day_of_week'] = fpc_df['date'].dt.dayofweek
fpc_df['hour_of_day'] = pd.to_datetime(fpc_df.MITG_CRT_LTS).dt.hour
fpc_df['tod'] = np.where(fpc_df.hour_of_day < 7, 'early', 'mid')
fpc_df['tod'] = np.where(fpc_df.hour_of_day > 19, 'late', fpc_df.tod)

#IMT Count grouped by time of day (TOD)
imt_count_tod = fpc_df.groupby(['tod', 'date']).MTC_ISS_TRKG_ID.count().reset_index().rename(columns = {'MTC_ISS_TRKG_ID':'y_tod'})

#Merge the tod counts with original df
df_merge = fpc_df.merge(imt_count_tod)

df_counts = df_merge.groupby('date')['y'].count().asfreq('D').reset_index()
df_counts = df_counts.fillna(method='ffill')
df_counts.rename(columns={'date':'ds'}, inplace=True) #Rename date column to the Prophet format

df_counts

,ds,y
0,2019-01-01,25.0
1,2019-01-02,21.0
2,2019-01-03,27.0
3,2019-01-04,30.0
4,2019-01-05,23.0
...,...,...
940,2021-07-29,24.0
941,2021-07-30,22.0
942,2021-07-31,12.0
943,2021-08-01,14.0
